# 📘 Performance Optimization: Maximize NLSQ Speed⏱️ **25-35 minutes** | 📊 **Level: ●●○ Intermediate** | 🏷️ **Performance**---

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/imewei/NLSQ/blob/main/examples/notebooks/02_core_tutorials/performance_optimization_demo.ipynb)


In [ ]:
# @title Install NLSQ (run once in Colab)
import sys
if 'google.colab' in sys.modules:
    print("Running in Google Colab - installing NLSQ...")
    !pip install -q nlsq
    print("✅ NLSQ installed successfully!")
else:
    print("Not running in Colab - assuming NLSQ is already installed")

## 🎯 What You'll Learn- ✓ **Profile** curve fitting performance- ✓ **Optimize** JAX compilation and JIT- ✓ **Leverage** GPU acceleration- ✓ **Minimize** memory overhead- ✓ **Apply** best practices for speed---

## Setup

In [1]:
# Configure matplotlib for inline plotting in VS Code/Jupyter
# MUST come before importing matplotlib
%matplotlib inline

In [2]:
import time

import jax
import jax.numpy as jnp
import numpy as np

from nlsq import CurveFit, curve_fit

devices = jax.devices()
print(f"Device: {devices[0].platform}")

Device: gpu


## 1. JAX Compilation Basics**Key insight:** First fit is slow (compilation), subsequent fits are fast

In [3]:
def exponential(x, a, b, c):
    return a * jnp.exp(-b * x) + c

x = np.linspace(0, 10, 1000)
y = 5 * np.exp(-0.5 * x) + 2 + np.random.normal(0, 0.1, 1000)

# First fit (slow - compilation)
start = time.time()
popt1, _ = curve_fit(exponential, x, y, p0=[4, 0.4, 1.5])
t1 = time.time() - start

# Second fit (fast - reuses compiled code)
start = time.time()
popt2, _ = curve_fit(exponential, x, y, p0=[4, 0.4, 1.5])
t2 = time.time() - start

print(f"First fit:  {t1:.4f}s (with compilation)")
print(f"Second fit: {t2:.4f}s (compiled)")
print(f"Speedup: {t1/t2:.1f}x")

INFO:nlsq.curve_fit:Starting curve fit | {'n_params': 3, 'n_data_points': 1000, 'method': 'trf', 'solver': 'auto', 'batch_size': None, 'has_bounds': False, 'dynamic_sizing': False}


INFO:nlsq.least_squares:Starting least squares optimization | {'method': 'trf', 'n_params': 3, 'loss': 'linear', 'ftol': 1e-08, 'xtol': 1e-08, 'gtol': 1e-08}


INFO:nlsq.optimizer.trf:Starting TRF optimization (no bounds) | {'n_params': 3, 'n_residuals': 1000, 'max_nfev': None}


PERFORMANCE:nlsq.optimizer.trf:Optimization: iter=0 | cost=1.630478e+02 | ‖∇f‖=1.112331e+03 | nfev=1


PERFORMANCE:nlsq.optimizer.trf:Optimization: iter=1 | cost=5.181930e+00 | ‖∇f‖=2.376997e+01 | step=4.290688e+00 | nfev=2


...

PERFORMANCE:nlsq.least_squares:Timer: optimization took 0.301639s


INFO:nlsq.least_squares:Convergence: reason=`ftol` termination condition is satisfied. | iterations=4 | final_cost=4.866939e+00 | time=0.302s | final_gradient_norm=3.256575073606882e-06


PERFORMANCE:nlsq.curve_fit:Timer: curve_fit took 0.540513s


INFO:nlsq.curve_fit:Curve fit completed | {'total_time': 0.5405130649996863, 'final_cost': 9.733877793989468, 'covariance_warning': False}


First fit:  1.9432s (with compilation)
Second fit: 0.6029s (compiled)
Speedup: 3.2x


## 2. Reusing CurveFit Objects**Best practice:** Reuse CurveFit objects to avoid recompilation

In [4]:
# Create reusable fitter
jcf = CurveFit()

# Multiple fits reuse compiled functions
times = []
for _ in range(10):
    start = time.time()
    popt, _ = jcf.curve_fit(exponential, x, y, p0=[4, 0.4, 1.5])
    times.append(time.time() - start)

print(f"Average fit time: {np.mean(times[1:]):.4f}s")
print("✓ Reusing CurveFit object eliminates compilation overhead")

INFO:nlsq.curve_fit:Starting curve fit | {'n_params': 3, 'n_data_points': 1000, 'method': 'trf', 'solver': 'auto', 'batch_size': None, 'has_bounds': False, 'dynamic_sizing': False}


INFO:nlsq.least_squares:Starting least squares optimization | {'method': 'trf', 'n_params': 3, 'loss': 'linear', 'ftol': 1e-08, 'xtol': 1e-08, 'gtol': 1e-08}


INFO:nlsq.optimizer.trf:Starting TRF optimization (no bounds) | {'n_params': 3, 'n_residuals': 1000, 'max_nfev': None}


PERFORMANCE:nlsq.optimizer.trf:Optimization: iter=0 | cost=1.630478e+02 | ‖∇f‖=1.112331e+03 | nfev=1


PERFORMANCE:nlsq.optimizer.trf:Optimization: iter=1 | cost=5.181930e+00 | ‖∇f‖=2.376997e+01 | step=4.290688e+00 | nfev=2


...

PERFORMANCE:nlsq.least_squares:Timer: optimization took 0.019456s


INFO:nlsq.least_squares:Convergence: reason=`ftol` termination condition is satisfied. | iterations=4 | final_cost=4.866939e+00 | time=0.019s | final_gradient_norm=3.256575073606882e-06


PERFORMANCE:nlsq.curve_fit:Timer: curve_fit took 0.101000s


INFO:nlsq.curve_fit:Curve fit completed | {'total_time': 0.10100040099860053, 'final_cost': 9.733877793989468, 'covariance_warning': False}


Average fit time: 0.0917s
✓ Reusing CurveFit object eliminates compilation overhead


## 3. Fixed Array Size**Issue:** Different array sizes trigger recompilation**Solution:** Use `flength` parameter

In [5]:
# Without fixed length (recompiles for each size)
jcf_dynamic = CurveFit()
sizes = [100, 200, 300]

for n in sizes:
    x_test = np.linspace(0, 10, n)
    y_test = 5 * np.exp(-0.5 * x_test) + 2
    start = time.time()
    popt, _ = jcf_dynamic.curve_fit(exponential, x_test, y_test, p0=[4, 0.4, 1.5])
    elapsed = time.time() - start
    print(f"Size {n}: {elapsed:.4f}s")

print("\n⚠️ Each size triggers recompilation")

INFO:nlsq.curve_fit:Starting curve fit | {'n_params': 3, 'n_data_points': 100, 'method': 'trf', 'solver': 'auto', 'batch_size': None, 'has_bounds': False, 'dynamic_sizing': False}


INFO:nlsq.least_squares:Starting least squares optimization | {'method': 'trf', 'n_params': 3, 'loss': 'linear', 'ftol': 1e-08, 'xtol': 1e-08, 'gtol': 1e-08}


INFO:nlsq.optimizer.trf:Starting TRF optimization (no bounds) | {'n_params': 3, 'n_residuals': 100, 'max_nfev': None}


PERFORMANCE:nlsq.optimizer.trf:Optimization: iter=0 | cost=1.631332e+01 | ‖∇f‖=1.099246e+02 | nfev=1


PERFORMANCE:nlsq.optimizer.trf:Optimization: iter=1 | cost=3.608026e-02 | ‖∇f‖=2.070611e+00 | step=4.290688e+00 | nfev=2


...

PERFORMANCE:nlsq.least_squares:Timer: optimization took 0.249072s


INFO:nlsq.least_squares:Convergence: reason=`gtol` termination condition is satisfied. | iterations=4 | final_cost=1.443399e-26 | time=0.249s | final_gradient_norm=3.544475795562745e-12


PERFORMANCE:nlsq.curve_fit:Timer: curve_fit took 0.504063s


INFO:nlsq.curve_fit:Curve fit completed | {'total_time': 0.5040625280016684, 'final_cost': 2.8867970396110317e-26, 'covariance_warning': False}


Size 300: 0.5732s

⚠️ Each size triggers recompilation


## 4. GPU vs CPU Performance**GPU advantage:** Increases with dataset size

In [6]:
if devices[0].platform == 'gpu':
    print("🚀 GPU Performance Test")
    sizes = [1000, 10000, 100000]

    for n in sizes:
        x_large = np.linspace(0, 10, n)
        y_large = 5 * np.exp(-0.5 * x_large) + 2 + np.random.normal(0, 0.1, n)
        start = time.time()
        popt, _ = curve_fit(exponential, x_large, y_large, p0=[4, 0.4, 1.5])
        elapsed = time.time() - start
        print(f"  {n:>6} points: {elapsed:.4f}s")
else:
    print("💻 CPU mode - GPU would provide 100-300x speedup")

INFO:nlsq.curve_fit:Starting curve fit | {'n_params': 3, 'n_data_points': 1000, 'method': 'trf', 'solver': 'auto', 'batch_size': None, 'has_bounds': False, 'dynamic_sizing': False}


INFO:nlsq.least_squares:Starting least squares optimization | {'method': 'trf', 'n_params': 3, 'loss': 'linear', 'ftol': 1e-08, 'xtol': 1e-08, 'gtol': 1e-08}


🚀 GPU Performance Test


INFO:nlsq.optimizer.trf:Starting TRF optimization (no bounds) | {'n_params': 3, 'n_residuals': 1000, 'max_nfev': None}


PERFORMANCE:nlsq.optimizer.trf:Optimization: iter=0 | cost=1.649197e+02 | ‖∇f‖=1.105774e+03 | nfev=1


...

PERFORMANCE:nlsq.least_squares:Timer: optimization took 0.339292s


INFO:nlsq.least_squares:Convergence: reason=`ftol` termination condition is satisfied. | iterations=4 | final_cost=5.021710e+02 | time=0.339s | final_gradient_norm=6.78356428807092e-08


PERFORMANCE:nlsq.curve_fit:Timer: curve_fit took 0.614294s


INFO:nlsq.curve_fit:Curve fit completed | {'total_time': 0.6142937969998457, 'final_cost': 1004.3420088628458, 'covariance_warning': False}


  100000 points: 0.6817s


## 🔑 Key Takeaways1. **Reuse CurveFit objects** to avoid recompilation2. **Use fixed array size** when processing multiple datasets3. **GPU acceleration** scales with dataset size4. **Warmup first fit** to compile functions5. **Profile before optimizing** - measure actual bottlenecks---

## 🔗 Next Steps- [Large Dataset Demo](large_dataset_demo.ipynb) - Memory optimization- [GPU Deep Dive](../03_advanced/gpu_optimization_deep_dive.ipynb) - Advanced GPU techniques- [Advanced Features](advanced_features_demo.ipynb) - Callbacks, robust fitting---